##  SALESIAN POLYTECHNIC UNIVERSITY 

**Members:** Pedro Bermeo, María José Peláez

**Subject:** Inteligencia Artificial II

### Digit Viewer and Identifier

Below is a detailed description of the process that was used to develop the digitizer and the results that were obtained.

### Explanation Code

* The code below allows you to read the corpus with the information of the numbers that you want to display on the screen of the matrix.

* The method **generate_indices ()** allows generating the indexes when opening the dataset.
* The method **get_digit ()** allows you to obtain the digits of the dataset and then list them in the combobox that is displayed on the screen.

In [ ]:
import re
import itertools

class Utilities:
    
    def __init__(self, path = './corpus/digits-database.data'):
        self.path = path
        self.regex = re.compile('(0|1){2,}')
        self.regexno = re.compile('(\s)+[0-9]{1}')
        
    
    def generate_indices(self):
        _dict = []
        with open(self.path, 'r') as _f:
            pivote = 0
            flag = False
            lineno = 0
            for line in _f:
                if self.regex.match(line)!=None and not flag:
                    pivote = lineno
                    flag = True
                if self.regexno.match(line)!=None and flag:
                    _dict.append((int(line.replace(' ','')),pivote,lineno))
                    flag = False
                lineno += 1
            _f.close()
            
        return _dict

    def get_digit(self,_slice, _end):
        data = []
        with open(self.path, 'r') as _f:
            for line in itertools.islice(_f, _slice, _end):
                data.append([int(i) for i in line.lstrip().rstrip()])
            
            _f.close()
        return data

* The method  **getCodification** allows the information received to be encoded and transformed using OneHotEncoder.

* The method **getTraining** allows training the neural network in this case it uses 1000 iterations with the activation **logistic**

* Also once the number drawn on the screen is recognized, it is possible to replace the zeros by the ones depending on the number of cells that were used.

* Next, we proceed to print the confusion matrix with the result of the precisions and the total average.

In [ ]:
	def obtenerCodificacion(self):
		salida = self.label_encoder.fit_transform(self.delta)
		onehot_encoder = OneHotEncoder(sparse=False)
		salida = salida.reshape(len(salida), 1)
		self.onehot_encoded = onehot_encoder.fit_transform(salida)
	
	def obtenerEntrenamiento(self):
	
		x_train, x_test, d_train, d_test = train_test_split(self.datos, self.onehot_encoded, test_size=0.80, random_state=0)
		mlp = MLPClassifier(solver = 'lbfgs', activation='logistic', verbose=True, alpha=1e-4, tol=1e-15, max_iter=1000, \
		hidden_layer_sizes=(1024, 512, 256, 128, 10))
		mlp.fit(self.datos, self.onehot_encoded)
		
		for entrada in self.datos:
			res = mlp.predict(entrada.reshape(1, -1))
			num = (np.argmax(res, axis=1)+1).reshape(-1, 1)

			aux = []
			matriz = []
			for i in range(32):
				for j in range(i * 32, (i + 1) * 32):
					aux.append(entrada[j])
				matriz.append(aux)
				aux = []

			for i in range(32):
				print(str(matriz[i]).replace(', ', ''))
			print(res, '=>',int(num[0] - 1))

		prediccion = (np.argmax(mlp.predict(x_test), axis = 1) + 1).reshape(-1, 1)
		print('Matriz de Confusion\n')
		matriz = confusion_matrix((np.argmax(d_test, axis = 1) + 1).reshape(-1, 1), prediccion)
		print(matriz)
		print('\n')
		print(classification_report((np.argmax(d_test, axis = 1) + 1).reshape(-1, 1), prediccion))

* The method shown below allows you to load the dataset, which we previously read, obtaining each of the digits and displaying them on the screen in order to make the corresponding selection and that the recognition can be observed on the screen.

In [ ]:
def cargaDatos(self, evt):
		self.renovar()
		_id = [int(i) for i in (str(self.cbxData.get()).replace(')','').replace(' ','').replace('(','')).split(',')]
		
		data = self.utilities.get_digit(_id[1],_id[2])
		for i in range(len(data)):
			for j in range(len(data[i])):
				if(data[i][j] == 1):
					self.seleccionar(j, i)

1. The display interface is presented below:
    
 ![Texto Alternativo](visualizador.png) 


2. By clicking on the button called **Training** the function that allows to perform is the training of the Neural network and allows to observe in console the training result and the confusion matrix.

**Iterations**

![Texto Alternativo](b.png) 

**Confusion matrix**

![Texto Alternativo](a.png) 

Below you can see the table with all the precisions and the average of these.

![Texto Alternativo](d.png)

3. Clicking on the button labeled **New** allows you to clean the matrix to redraw a number.

4. Clicking the button called **Draw Number** allows once a number has been drawn on the screen to obtain the prediction and allows me to obtain which number is the one that draws that through the training that the neural network had for that is more precise.

### Results:

* We are going to draw the number one in the matrix and we will observe the result that the prediction of the neural network returns us.
* The result that was obtained was also one, so the prediction is correct, as seen in the image.

![Texto Alternativo](e.png) 

* It can be seen that in the console the ones represent what was drawn on the screen and the zeros represent the cells that were not used.

![Texto Alternativo](c.png) 

* We are going to draw the number two in the matrix and we will observe the result that the prediction of the neural network returns us.
* The result that was obtained was also two, so the prediction is correct, as seen in the image.

![Texto Alternativo](img4.png)

* We are going to draw the number three in the matrix and we will observe the result that the prediction of the neural network returns us.
* The result that was obtained was also three, so the prediction is correct, as seen in the image.

![Texto Alternativo](img6.png)

* We are going to draw the number four in the matrix and we will observe the result that the prediction of the neural network returns us.
* The result that was obtained was also four, so the prediction is correct, as seen in the image.

![Texto Alternativo](img2.png)

* We are going to draw the number five in the matrix and we will observe the result that the prediction of the neural network returns us.
* The result that was obtained was also five, so the prediction is correct, as seen in the image.

![Texto Alternativo](img1.png)

* We are going to draw the number six in the matrix and we will observe the result that the prediction of the neural network returns us.
* The result that was obtained was also six, so the prediction is correct, as seen in the image.

![Texto Alternativo](img7.png)

* We are going to draw the number seven in the matrix and we will observe the result that the prediction of the neural network returns us.
* The result that was obtained was also seven, so the prediction is correct, as seen in the image.

![Texto Alternativo](img5.png)

* We are going to draw the number nine in the matrix and we will observe the result that the prediction of the neural network returns us.
* The result that was obtained was also nine, so the prediction is correct, as seen in the image.

![Texto Alternativo](img3.png)


* The result that is displayed next is the reading of the dataset, which was read the data and the corresponding number of the dataset that we have selected through the combobox is displayed on the screen.

![Texto Alternativo](f.png)

[Link of Video](https://drive.google.com/open?id=1JLg8_MTpUZIlgEfOdx6VQx1-ib4rC7_v)
    

    
    